# Conversational RAG

In [18]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4 unstructured 

Note: you may need to restart the kernel to use updated packages.


In [19]:
import getpass
import os
import langchain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_chroma import Chroma
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub
from langchain_core.runnables.history import RunnableWithMessageHistory


## 1. Files Loading

In [20]:
path = r"C:\Users\reply\Documents\GitHub\RepoAgent\markdown_docs\repo_agent"
loader = DirectoryLoader(path, glob="./*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader)
docs = loader.load()

  0%|          | 0/14 [00:00<?, ?it/s]

100%|██████████| 14/14 [00:05<00:00,  2.64it/s]


In [21]:
docs[0].metadata

{'source': 'C:\\Users\\reply\\Documents\\GitHub\\RepoAgent\\markdown_docs\\repo_agent\\change_detector.md'}

In [2]:
import os

In [5]:
#import directory loader
from langchain_community.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader

In [13]:
print(all_docs)
len(all_docs)

[Document(metadata={'source': 'C:\\Users\\reply\\RepoAgent\\markdown_docs\\display\\book_tools\\generate_repoagent_books.md'}, page_content="FunctionDef main\n\nmain: The function of main is to copy Markdown documentation files from a specified folder to a destination folder, create a README.md file if it does not exist, and organize the copied files accordingly.\n\nparameters:\n- markdown_docs_folder: The folder containing the Markdown documentation files.\n- book_name: The name of the book being generated.\n- repo_path: The path to the repository.\n\nCode Description:\nThe main function first creates a destination directory for the book by joining the book name with the 'src' folder inside the './books' directory. It then determines the source directory by joining the repo_path with the markdown_docs_folder.\n\nIf the destination directory does not exist, it creates one and prints a message indicating the creation. It then iterates through the items in the source directory. For each 

41

In [12]:
path = "C:\\Users\\reply\\RepoAgent\\markdown_docs"
all_docs = []
for subdir, _, _ in os.walk(path):
            try:
                # Instantiate the loader for each subdirectory
                loader = DirectoryLoader(subdir, glob="*.md", show_progress=False, loader_cls=UnstructuredMarkdownLoader)
                docs = loader.load()
                print(docs)

                all_docs.extend(docs)
            except Exception as e:
                print(f"Error loading documents from {subdir}: {e}")



[]
[]
[Document(metadata={'source': 'C:\\Users\\reply\\RepoAgent\\markdown_docs\\display\\book_tools\\generate_repoagent_books.md'}, page_content="FunctionDef main\n\nmain: The function of main is to copy Markdown documentation files from a specified folder to a destination folder, create a README.md file if it does not exist, and organize the copied files accordingly.\n\nparameters:\n- markdown_docs_folder: The folder containing the Markdown documentation files.\n- book_name: The name of the book being generated.\n- repo_path: The path to the repository.\n\nCode Description:\nThe main function first creates a destination directory for the book by joining the book name with the 'src' folder inside the './books' directory. It then determines the source directory by joining the repo_path with the markdown_docs_folder.\n\nIf the destination directory does not exist, it creates one and prints a message indicating the creation. It then iterates through the items in the source directory. For

## 2. Split and Preprocess

In [22]:
def split_documents(doc, chunk_size=250, chunk_overlap=30):
    headers_to_split_on = [
         ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=False
    )
    md_header_splits = markdown_splitter.split_text("#"+ os.path.basename(doc.metadata['source']) + " \n\n  " + doc.page_content)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    # Split
    splits = text_splitter.split_documents(md_header_splits)
   
    return splits

In [35]:
all_splits = []

In [23]:
all_splits = []
for doc in docs:
    splits = split_documents(doc)
    all_splits.extend(splits)


In [24]:
splits[0]

Document(page_content='#settings.md  \nClassDef LogLevel  \nLogLevel: The function of LogLevel is to define different log levels such as DEBUG, INFO, WARNING, ERROR, and CRITICAL.\nattributes:\n- DEBUG: "DEBUG"\n- INFO: "INFO"\n- WARNING: "WARNING"\n- ERROR: "ERROR"')

In [25]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


## 3. Prompting

In [26]:
model = ChatOpenAI(model="gpt-3.5-turbo-0125")



In [27]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks regarding code documentation file. "
    "Use the following pieces of retrieved context to answer "
    "the question. It's also specified the name of the file that contains the functions.  If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

## 4. Test

In [28]:
def paginate_text(text, page_size=500):
    return [text[i:i + page_size] for i in range(0, len(text), page_size)]


In [34]:
query = input("Enter your query (or 'exit' to quit): ")
response = conversational_rag_chain.invoke(
    {"input": query},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]
print(f"Question: {query}\n")
paginated_response = paginate_text(response)

for i, page in enumerate(paginated_response):
    print(f"Page {i + 1}/{len(paginated_response)}: {page}")
    if i < len(paginated_response) - 1:
            input("Press Enter to continue to the next page...")

print("\n")

Question: doc meta info

Page 1/1: The content of the meta-info.json file includes information such as the author, version, and description of the code documentation. This file is used to provide metadata about the codebase and its documentation.




In [30]:
splits[0]

Document(page_content='#settings.md  \nClassDef LogLevel  \nLogLevel: The function of LogLevel is to define different log levels such as DEBUG, INFO, WARNING, ERROR, and CRITICAL.\nattributes:\n- DEBUG: "DEBUG"\n- INFO: "INFO"\n- WARNING: "WARNING"\n- ERROR: "ERROR"')

In [31]:
rag_chain.invoke({"input":"how get_staged_pys works?"})

KeyError: "Input to ChatPromptTemplate is missing variables {'chat_history'}.  Expected: ['chat_history', 'context', 'input'] Received: ['input', 'context']"

In [ ]:
conversational_rag_chain.invoke(
   {"input":"no"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"I don't have the specific information about the markdown file where the behavior of the last_element function is explained."

In [ ]:
rag_chain.invoke({"input":"What file named last does?"})

KeyError: "Input to ChatPromptTemplate is missing variables {'chat_history'}.  Expected: ['chat_history', 'context', 'input'] Received: ['input', 'context']"

In [ ]:
rag_chain.invoke("What last.md does?")

'last.md saves and updates the markdown content of an existing file after making structural changes.'

In [ ]:
rag_chain.invoke("What prova.md does?")

'The prova.md function assigns values to variables a, b, and c, and then returns the values of b and c. It does not take any parameters and is used for version control operations with the git library.'